# Introduction

This notebook will illustrate experiments with the Adam optimizer in TensorFlow to solve the `pde_01` 2-D PDE IVP.

## The `pde_01` PDE problem

This PDE was created as a sanity check for the code. The differential equation is provided in the form:

\begin{equation}
    \nabla \psi (x,y) = y^2 + 2 x y
\end{equation}

or:

\begin{equation}
    G \left( x, y, \psi, \frac {\partial \psi} {dx}, \frac {\partial \psi} {\partial y} \right) = \frac {\partial \psi} {\partial x} + \frac {\partial \psi} {dy} - y^2 - 2 x y = 0
\end{equation}

with initial conditions:

\begin{equation}
    f_0(0, y) = \psi(0, y) = 0 \\
    g_0(x, 0) = \psi(x, 0) = 0 \\
\end{equation}

The analytical solution is:

\begin{equation}
    \psi_a(x, y) = x y^2
\end{equation}

The derivatives are:

\begin{equation}
    \frac {\partial \psi} {\partial x} = y^2 \\
    \frac {\partial \psi} {\partial y} = 2 x y
\end{equation}

The analytical solution in the domain $0 \leq x,y \leq 1$ is plotted below.

In [ ]:
# Import standard Python modules.
import datetime
import importlib
from math import exp
import os
import platform
import sys

# Import 3rd-party modules.
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

# Import TensorFlow.
import tensorflow as tf

In [ ]:
# Import the problem definition.
eq_name = "pde_01"
eq_module = 'nnde.differentialequation.examples.pde_01'
eq = importlib.import_module(eq_module)

# Compute the analytical solution.
# Note that Y is used in place of \psi in the code.
na = 101
xa = np.linspace(0, 1, na)
ya = np.linspace(0, 1, na)
Ya = [[eq.Ya([x, y]) for x in xa] for y in ya]
Ya = np.array(Ya)

# Plot the analytical solution.
fig = plt.figure()
(X, Y) = np.meshgrid(xa, ya)
ax = Axes3D(fig, auto_add_to_figure=False)
fig.add_axes(ax)
ax.plot_surface(X, Y, Ya)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('$\psi(x,y)$')
plt.title("Analytical solution for %s" % eq_name)
plt.show()

## Solving `pde_01` using TensorFlow

In [ ]:
def print_system_information():
    print("System report:")
    print(datetime.datetime.now())
    print("Host name: %s" % platform.node())
    print("OS: %s" % platform.platform())
    print("uname:", platform.uname())
    print("Python version: %s" % sys.version)
    print("Python build:", platform.python_build())
    print("Python compiler: %s" % platform.python_compiler())
    print("Python implementation: %s" % platform.python_implementation())
    # print("Python file: %s" % __file__)

In [ ]:
def create_output_directory(path=None):
    path_noext, ext = os.path.splitext(path)
    output_dir = path_noext
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    return output_dir

In [ ]:
from nnde.math.trainingdata import create_training_grid2

def create_training_data(*n_train):
    x_train = np.array(create_training_grid2(*n_train))
    return x_train

In [ ]:
def build_model(H, w0_range, u0_range, v0_range):
    hidden_layer = tf.keras.layers.Dense(
        units=H, use_bias=True,
        activation=tf.keras.activations.sigmoid,
        kernel_initializer=tf.keras.initializers.RandomUniform(*w0_range),
        bias_initializer=tf.keras.initializers.RandomUniform(*u0_range)
    )
    output_layer = tf.keras.layers.Dense(
        units=1,
        activation=tf.keras.activations.linear,
        kernel_initializer=tf.keras.initializers.RandomUniform(*v0_range),
        use_bias=False,
    )
    model = tf.keras.Sequential([hidden_layer, output_layer])
    return model

In [ ]:
print_system_information()

In [ ]:
# Define the hyperparameters.

# Set up the output directory.
path = "./pde_01_tf_adam"
output_dir = create_output_directory(path)

# Training optimizer
training_algorithm = "Adam"

# Initial parameter ranges
w0_range = [-0.1, 0.1]
u0_range = [-0.1, 0.1]
v0_range = [-0.1, 0.1]

# Number of hidden nodes.
H = 10

# Number of training points in each dimension.
nx_train = 11
ny_train = 11
n_train = nx_train*ny_train

# Number of training epochs.
n_epochs = 1000

# Learning rate.
learning_rate = 0.01

# Random number generator seed.
random_seed = 0

In [ ]:
# Create and save the training data.
xy_train = create_training_data(nx_train, ny_train)
x_train = xy_train[::ny_train, 0]
y_train = xy_train[:nx_train, 1]
np.savetxt(os.path.join(output_dir,'xy_train.dat'), xy_train)

In [ ]:
# Build the model.
model = build_model(H, w0_range, u0_range, v0_range)

# Create the optimizer.
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
# Train the model.

# Create history variables.
losses = []
phist = []

# Set the random number seed for reproducibility.
tf.random.set_seed(random_seed)

# Rename the training data Variable (_v) for convenience, just for training.
xy_train_v = tf.Variable(xy_train, dtype=tf.float32, name="xy_train")
xy = xy_train_v

print("Hyperparameters: n_train = %s, H = %s, n_epochs = %s, learning_rate = %s"
      % (n_train, H, n_epochs, learning_rate))
t_start = datetime.datetime.now()
print("Training started at", t_start)

for epoch in range(n_epochs):

    # Run the forward pass.
    with tf.GradientTape() as tape2:
        with tf.GradientTape() as tape1:

            # Compute the network output.
            N = model(xy)

            # Compute the trial solution.
            # A = 0
            Y = xy[:, 0]*xy[:, 1]*N[:, 0]

        # Compute the first partial derivatives.
        dY_dxy = tape1.gradient(Y, xy)

        # Compute the differential equation estimate.
        G = dY_dxy[:, 0] + dY_dxy[:, 1] - xy[:, 1]**2 - 2*xy[:, 0]*xy[:, 1]

        # Compute the loss function.
        L = tf.math.sqrt(tf.reduce_sum(G**2)/n_train)
        losses.append(L.numpy())
    
    # Compute the gradient of the loss function wrt the model parameters.
    pgrad = tape2.gradient(L, model.trainable_variables)

    # Save the parameters used in this pass.
    phist.append(
        np.hstack(
            (model.trainable_variables[0].numpy().reshape((2*H,)),    # w (2, H) matrix -> (2H,) row vector
             model.trainable_variables[1].numpy(),       # u (H,) row vector
             model.trainable_variables[2][:, 0].numpy()) # v (H, 1) column vector
        )
    )

    # Update the parameters for this pass.
    optimizer.apply_gradients(zip(pgrad, model.trainable_variables))

t_stop = datetime.datetime.now()
print("Training stopped at", t_stop)
t_elapsed = t_stop - t_start
print("Total training time was %s seconds." % t_elapsed.total_seconds())

# Save the parameter histories.
np.savetxt(os.path.join(output_dir, 'phist.dat'), np.array(phist))

In [ ]:
# Compute and save the trained results at training points.
with tf.GradientTape() as tape1:
    N = model(xy)
    # A = 0
    Yt_train = xy[:, 0]*xy[:, 1]*N[:, 0]
dYt_dx_train = tape1.gradient(Yt_train, xy)
np.savetxt(os.path.join(output_dir, 'Yt_train.dat'), Yt_train.numpy().reshape((n_train)))
np.savetxt(os.path.join(output_dir, 'dYt_dx_train.dat'), dYt_dx_train[:, 0].numpy().reshape((n_train)))
np.savetxt(os.path.join(output_dir, 'dYt_dy_train.dat'), dYt_dx_train[:, 1].numpy().reshape((n_train)))

# Compute and save the analytical solution and derivative at training points.
Ya_train = np.array([eq.Ya(xy) for xy in xy_train])
np.savetxt(os.path.join(output_dir,'Ya_train.dat'), Ya_train)
dYa_dx_train = np.array([eq.dYa_dx(xy) for xy in xy_train])
dYa_dy_train = np.array([eq.dYa_dy(xy) for xy in xy_train])
np.savetxt(os.path.join(output_dir,'dYa_dx_train.dat'), dYa_dx_train)
np.savetxt(os.path.join(output_dir,'dYa_dy_train.dat'), dYa_dy_train)

# Compute and save the error in the trained solution and derivatives at training points.
Yt_err_train = Yt_train.numpy().reshape((n_train)) - Ya_train
np.savetxt(os.path.join(output_dir, 'Yt_err_train.dat'), Yt_err_train)
dYt_dx_err_train = dYt_dx_train[:, 0].numpy().reshape((n_train)) - dYa_dx_train
dYt_dy_err_train = dYt_dx_train[:, 1].numpy().reshape((n_train)) - dYa_dy_train
np.savetxt(os.path.join(output_dir, 'dYt_dx_err_train.dat'), dYt_dx_err_train)
np.savetxt(os.path.join(output_dir, 'dYt_dy_err_train.dat'), dYt_dy_err_train)

In [ ]:
# Plot the loss function history.
plt.semilogy(losses, label="L")
plt.xlabel("Epoch")
plt.ylabel("Loss function ($\sqrt {\sum G_i^2/N}$)")
plt.grid()
plt.legend()
plt.title("Loss function (RMS error) evolution for %s (%s, %s)\n$\eta$=%s, H=%s, $n_{train}$=%s, $w_0$=%s, $u_0$=%s, $v_0$=%s" %
          (eq_name, 'TensorFlow', training_algorithm, learning_rate, H, n_train, w0_range, u0_range, v0_range))
plt.show()

In [ ]:
# Plot the error in the trained solution at the training points.
fig = plt.figure()
(X, Y) = np.meshgrid(x_train, y_train)
Z = Yt_err_train.reshape((nx_train, ny_train))
ax = Axes3D(fig, auto_add_to_figure=False)
fig.add_axes(ax)
ax.plot_surface(X, Y, Z)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('$\psi_t(x,y)-\psi_a(x,y)$')
plt.show()

In [ ]:
# Plot the parameter histories.
phist = np.array(phist)
plt.figure(figsize=(12, 14))

# w
plt.subplot(311)
plt.plot(phist[:, 0:H])
plt.title("Hidden weight $w$")
plt.grid()

# u
plt.subplot(312)
plt.plot(phist[:, H:2*H])
plt.title("Hidden bias $u$")
plt.grid()

# v
plt.subplot(313)
plt.plot(phist[:, 2*H:3*H])
plt.title("Output weight $v$")
plt.grid()

plt.suptitle("Parameter evolution for model 1 %s (%s, %s)\n$\eta$=%s, H=%s, $n_{train}$=%s, $w_0$=%s, $u_0$=%s, $v_0$=%s" %
          (eq_name, 'TF', training_algorithm, learning_rate, H, n_train, w0_range, u0_range, v0_range))
plt.subplots_adjust(hspace=0.2)
plt.show()